In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


dataset_folder_name = './/Screen_Capturing/Data'
TRAIN_TEST_SPLIT = 0.8
IM_WIDTH = 198
IM_HEIGHT = 198
IMAGE_SAMPLE_SIZE = 7800

Retrieve Data

In [ ]:
replace_dict = {"'mouse'":'"mouse"',"'click'":'"click"',"'keyboard'":'"keyboard"',"['":'["',"', '":'", "', "']":'"]'}

def clean_input(input):
    for key, value in replace_dict.items():
        input = input.replace(key, value)
    return input

def parse_dataset():
    with open(os.path.join(dataset_folder_name,'data.txt'), 'r', encoding="utf-8") as file:
        files_data = file.read().split("\n")
        
    records = []
    for file_data in files_data[: min(IMAGE_SAMPLE_SIZE, len(files_data)-1)]:
        file_name, inputs = file_data.split("\t")
        inputs = clean_input(inputs)
        inputs = json.loads(inputs)
        file_name = file_name + ""
        data = inputs["mouse"][0], inputs["mouse"][1], inputs["click"][0], inputs["keyboard"], file_name
        records.append(data)

    return records

In [ ]:
def pd_dataset(dataset):
    df = pd.DataFrame(dataset)
    df.columns = ['x', 'y', 'click', 'keyboard', 'file']

    return df

In [ ]:
dataset = parse_dataset()
df = pd_dataset(dataset)
df.head(100)

Display Data

In [ ]:
binwidth = 10

x_values = df['x'].values.tolist()
y_values = df['y'].values.tolist()

plt.hist(df['x'],bins=range(min(df['x']), 1920 + binwidth, binwidth))
plt.hist(df['y'],bins=range(min(df['y']), 1080 + binwidth, binwidth))
plt.show()

In [ ]:
plt.hist(df['click'])
plt.show()

In [ ]:
from scipy.ndimage import gaussian_filter
import matplotlib.cm as cm

In [ ]:
def myplot(s,bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x_values, y_values, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)

    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent


s = 64
img, extent = myplot(s)
plt.imshow(img, extent=extent, origin='lower', cmap=cm.jet)

In [ ]:
df['keyboard'].value_counts()[:10]

In [ ]:
df

Cleaning Data

In [ ]:
for index, key in df.iterrows():
    print(index)

In [ ]:
valid_keys_list = ['w','s','a','d','shift','e','q','1','2','r']
valid_click_list = ["Button.left","Button.right"]

data = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

def clean_data(df):
    
    records = []

    for index, key in df.iterrows():
        data = [0] * 10
        keyboard_keys = str(key['keyboard'])
        keyboard_keys = keyboard_keys.replace("'", '"')
        keyboard_keys = json.loads(keyboard_keys)

        data.insert(0, int(key['x']))
        data.insert(1, int(key['y']))

        for i, k in enumerate(valid_keys_list):
            if k in keyboard_keys:
                data[i+2] = 1

        if valid_click_list[0] in key['click']:
            data.insert(2, 1)
        else:
            data.insert(2, 0)

        if valid_click_list[1] in key['click']:
            data.insert(3, 1)
        else:
            data.insert(3, 0)

        data.append(key['file'])

        records.append(data)
    return records

def pd_labels(dataset):
    df = pd.DataFrame(dataset)
    df.columns = ['x','y','left_click','right_click','w','s','a','d','shift','e','q','1','2','r','file']

    return df

dataset = clean_data(df)
df = pd_labels(dataset)
df.head(100)

Editing Data to # of occurrence

In [ ]:
def add_key(key, dict):
    key = str(key)
    key = key.replace("'", '"')
    key = json.loads(key)
    if not key:
        if '' not in dict:
            dict[''] = 1
        else:
            dict[''] += 1
    for i in range(len(key)):
        if key[i] not in dict:
            dict[key[i]] = 1
        else:
            dict[key[i]] += 1
    return dict

In [ ]:
keyboard_data = {}

for i in df['keyboard']:
    keyboard_data = add_key(i, keyboard_data)

keyboard_data = {k: v for k, v in sorted(keyboard_data.items(), key=lambda item: item[1], reverse=True)}

keyboard_data

Display # of occurances of keys

In [ ]:
keys = list(keyboard_data.keys())
values = list(keyboard_data.values())

plt.rcParams['figure.figsize'] = [10, 10]
plt.bar(keys[:15], values[:15], log=1)

Truncate Data to the top 10 most occured keys

In [ ]:
import itertools
dataset_dict = {}
dataset = {}
for i in range(len(keyboard_data)):
    dataset[i] = keys[i]

dataset_dict["keyboard"] = dict(itertools.islice(dataset.items(), 0, 10))

dataset_dict

Clicks dict

In [ ]:
def add_click(click, dict):
    click = str(click)
    click = click.replace("'", '"')
    if not click:
        if '' not in dict:
            dict[''] = 1
        else:
            dict[''] += 1
    elif click not in dict:
        dict[click] = 1
    else:
        dict[click] += 1
    return dict

In [ ]:
click_data = {}

for i in df['click']:
    click_data = add_click(i, click_data)

click_data = {k: v for k, v in sorted(click_data.items(), key=lambda item: item[1], reverse=True)}

dataset_dict["click"] = click_data
dataset_dict

In [ ]:
for types in dataset_dict:
    dataset_dict[types] = dict((g, i) for i, g in dataset_dict[types].items())

dataset_dict

In [ ]:
max_x = df['x'].max()
max_y = df['y'].max()

Image Generator

In [ ]:
from keras.utils import to_categorical
from PIL import Image


class DataGenerator():
    def __init__(self, df):
        self.df = df

    def generate_split_indexes(self):
        p = np.random.permutation(len(df))
        print(p)
        train_up_to = int(len(df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]


        self.max_x = df['x'].max()
        self.max_y = df['y'].max()
        return train_idx, valid_idx, test_idx

    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open(img_path).convert('RGB')
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        batch_x = []
        batch_y = []
        while True:
            batch_x = []
            batch_y = []
            
            for idx in image_idx:
                img_path = self.df.iloc[idx]['file'] + '.jpeg'
                img_path = os.path.join(dataset_folder_name, img_path)
                img = self.preprocess_image(img_path)
                x = [self.df.iloc[idx][col]/max_x if col in ['x', 'y'] else self.df.iloc[idx][col] 
                    for col in ['x', 'y', 'left_click', 'right_click', 'w', 's', 'a', 'd', 'shift', 'e', 'q', '1', '2', 'r']]
                
                batch_x.append(img)
                batch_y.append(x)
                
                if len(batch_x) == batch_size:
                    yield np.array(batch_x), np.array(batch_y)
                    batch_x = []
                    batch_y = []
                    
            if not is_training:
                break

data_generator = DataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()

In [ ]:
from keras.utils import to_categorical
from PIL import Image


class DataGenerator():
    def __init__(self, df):
        self.df = df

    def generate_split_indexes(self):
        p = np.random.permutation(len(df))
        print(p)
        train_up_to = int(len(df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]


        max_x = df['x'].max()
        max_y = df['y'].max()
        return train_idx, valid_idx, test_idx

    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        im = Image.open(img_path).convert('RGB')
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    def generate_images(self, image_idx, is_training, batch_size=16):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """

        x_list = []
        y_list = []
        left_click_list = []
        right_click_list = []
        w_list = []
        s_list = []
        a_list = []
        d_list = []
        shift_list = []
        e_list = []
        q_list = []
        one_list = []
        two_list = []
        r_list = []
        images = []


        while True:
            for idx in image_idx:

                img_path = self.df.iloc[idx]['file']
                img = self.preprocess_image(img_path)
                x = self.df.iloc[idx]['x']
                y = self.df.iloc[idx]['y']
                left_click = self.df.iloc[idx]['left_click']
                right_click = self.df.iloc[idx]['right_click']
                w = self.df.iloc[idx]['w']
                s = self.df.iloc[idx]['s']
                a = self.df.iloc[idx]['a']
                d = self.df.iloc[idx]['d']
                shift = self.df.iloc[idx]['shift']
                e = self.df.iloc[idx]['e']
                q = self.df.iloc[idx]['q']
                one = self.df.iloc[idx]['1']
                two = self.df.iloc[idx]['2']
                r = self.df.iloc[idx]['r']
                

                x_list.append(x / max_x)
                y_list.append(y / max_y)
                left_click_list.append(left_click)
                right_click_list.append(right_click)
                w_list.append(w)
                s_list.append(s)
                a_list.append(a)
                d_list.append(d)
                shift_list.append(shift)
                e_list.append(e)
                q_list.append(q)
                one_list.append(one)
                two_list.append(two)
                r_list.append(r)
                images.appendd(img)

                if len(images) == batch_size:
                    yield np.array(images), [np.array(images), np.array(x_list), np.array(y_list), np.array(left_click_list), np.array(right_click_list), np.array(w_list), np.array(s_list), np.array(a_list), np.array(d_list), np.array(shift_list), np.array(e_list), np.array(q_list), np.array(one_list), np.array(two_list), np.array(r_list)]
                    x_list = []
                    y_list = []
                    left_click_list = []
                    right_click_list = []
                    w_list = []
                    s_list = []
                    a_list = []
                    d_list = []
                    shift_list = []
                    e_list = []
                    q_list = []
                    one_list = []
                    two_list = []
                    r_list = []
                    images = []

            if not is_training:
                break

data_generator = DataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()

In [ ]:
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=16)
train_gen

In [ ]:
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf

class model():
    """
    Used to define the model architecture.
    """

    def conv_block(self, inputs, filters, kernel_size, padding="same"):
        x = Conv2D(filters, kernel_size,padding=padding)(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        return x

    def dense_block(self, inputs, units, activation="relu"):
        x = Dense(units)(inputs)
        x = Activation(activation)(x)
        x = BatchNormalization(axis=-1)(x)
        x = Dropout(0.5)(x)
        return x

    def default_layers(self, inputs):
        x = self.conv_block(inputs, 16, (3, 3))
        x = self.conv_block(x, 32, (3, 3))
        x = self.conv_block(x, 32, (3, 3))
        return x

    def build_x_position_branch(self, inputs, num_outputs=1):
        x = self.default_layers(inputs)
        x = Flatten()(x)
        x = self.dense_block(x, 128)
        x = Dense(num_outputs)(x)
        x = Activation("linear", name='x')(x)
        return x

    def build_y_position_branch(self, inputs, num_outputs=1):
        x = self.default_layers(inputs)
        x = Flatten()(x)
        x = self.dense_block(x, 128)
        x = Dense(num_outputs)(x)
        x = Activation("linear", name='y')(x)
        return x

    def build_action_branch(self, inputs, name, num_outputs=1):
        x = self.default_layers(inputs)
        x = Flatten()(x)
        x = self.dense_block(x, 128)
        x = Dense(num_outputs)(x)
        x = Activation("sigmoid", name=name)(x)
        return x

    def build_softmax_branch(self, inputs, name, num_outputs=2):
        x = self.default_layers(inputs)
        x = Flatten()(x)
        x = self.dense_block(x, 128)
        x = Dense(num_outputs)(x)
        x = Activation("softmax", name=name)(x)
        return x

    def build_model(self, width, height):
        inputs = Input(shape=(height, width, 3))
        x_position = self.build_x_position_branch(inputs)
        y_position = self.build_y_position_branch(inputs)
        left_click = self.build_action_branch(inputs, "left_click")
        right_click = self.build_action_branch(inputs, "right_click")
        w = self.build_action_branch(inputs, "w")
        s = self.build_action_branch(inputs, "s")
        a = self.build_action_branch(inputs, "a")
        d = self.build_action_branch(inputs, "d")
        shift = self.build_action_branch(inputs, "shift")
        e = self.build_action_branch(inputs, "e")
        q = self.build_action_branch(inputs, "q")
        one = self.build_action_branch(inputs, "one")
        two = self.build_action_branch(inputs, "two")
        r = self.build_action_branch(inputs, "r")


        model = Model(inputs=inputs, outputs=[x_position, y_position, left_click, right_click, w, s, a, d, shift, e, q, one, two, r], name='bot')
        return model

    def build_soft_model(self, width, height):
        inputs = Input(shape=(height, width, 3))
        x_position = self.build_x_position_branch(inputs)
        y_position = self.build_y_position_branch(inputs)
        click = self.build_softmax_branch(inputs, "click")
        for_back_move = self.build_action_branch(inputs, "for_back_move")
        left_right_move = self.build_action_branch(inputs, "left_right_move")
        shift = self.build_action_branch(inputs, "shift")
        e = self.build_action_branch(inputs, "e")
        q = self.build_action_branch(inputs, "q")
        selection = self.build_softmax_branch(inputs, "selection")
        r = self.build_action_branch(inputs, "r")

        model = Model(inputs=inputs, outputs=[x_position, y_position, click, for_back_move, left_right_move, shift, e, q, selection, r], name='bot')
        return model

model = model().build_model(IM_WIDTH, IM_HEIGHT)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
from keras.optimizers import Adam
init_lr = 1e-4
epochs = 10
opt = Adam(lr=init_lr, decay=init_lr / epochs)

actions = ['left_click','right_click','w','s','a','d','shift','e','q','one','two','r']

loss = {'x': 'mse', 'y': 'mse'}
for index, key in enumerate(actions):
    loss[key] = 'binary_crossentropy'

loss_weights = {'x': 1.0, 'y': 1.0}
for index, key in enumerate(actions):
    loss_weights[key] = 1.5

metrics = {'x': 'mae', 'y': 'mae'}
for index, key in enumerate(actions):
    metrics[key] = 'accuracy'

model.compile(optimizer=opt, 
              loss=loss,
              loss_weights=loss_weights,
              metrics=metrics)

In [ ]:
from keras.callbacks import ModelCheckpoint
batch_size = 32
valid_batch_size = 8
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]
history = model.fit(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

In [ ]:
model.save('model.h5')

In [ ]:
color_map = plt.cm.plasma
for layers in model.layers:
    if str(layers.name).find("lstm") != -1:
        weights,hidden, bias = layers.get_weights()
        plt.matshow(weights, fignum=200, cmap=color_map)
        plt.show()
    if str(layers.name).find("dense") != -1:
        weights, bias = layers.get_weights()
        plt.matshow(weights, fignum=200, cmap=color_map)
        plt.show()

In [ ]:
xpoints = list(range(0,10))
for category in history.history:
    if category.find("val") != -1:
        if category.find("loss") != -1:
            if category == "val_loss":
                continue
            ypoints = history.history[category]
            plt.plot(xpoints, ypoints, label = category)

plt.legend()

In [ ]:
validation_steps=len(valid_idx)//valid_batch_size
test_batch_size = 16
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
x_pred, y_pred, left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred = model.predict(test_generator, steps=len(test_idx)//test_batch_size)

In [ ]:
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
samples = 1
images, x_true, y_true, left_click_true, right_click_true, w_true, s_true, a_true, d_true, shift_true, e_true, q_true, one_true, two_true, r_true = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    x_true.extend(labels[0])
    y_true.extend(labels[1])
    left_click_true.extend(labels[2])
    right_click_true.extend(labels[3])
    w_true.extend(labels[4])
    s_true.extend(labels[5])
    a_true.extend(labels[6])
    d_true.extend(labels[7])
    shift_true.extend(labels[8])
    e_true.extend(labels[9])
    q_true.extend(labels[10])
    one_true.extend(labels[11])
    two_true.extend(labels[12])
    r_true.extend(labels[13])

    
x_true = np.array(x_true)
y_true = np.array(y_true)
left_click_true = np.array(left_click_true)
right_click_true = np.array(right_click_true)
w_true = np.array(w_true)
s_true = np.array(s_true)
a_true = np.array(a_true)
d_true = np.array(d_true)
shift_true = np.array(shift_true)
e_true = np.array(e_true)
q_true = np.array(q_true)
one_true = np.array(one_true)
two_true = np.array(two_true)
r_true = np.array(r_true)

x_true, y_true = x_true * data_generator.max_x, y_true * data_generator.max_y
x_pred, y_pred = x_pred * data_generator.max_x, y_pred * data_generator.max_y

left_click_true, right_click_true, w_true, s_true, a_true, d_true, shift_true, e_true, q_true, one_true, two_true, r_true = left_click_true.argmax(axis=-1), right_click_true.argmax(axis=-1), w_true.argmax(axis=-1), s_true.argmax(axis=-1), a_true.argmax(axis=-1), d_true.argmax(axis=-1), shift_true.argmax(axis=-1), e_true.argmax(axis=-1), q_true.argmax(axis=-1), one_true.argmax(axis=-1), two_true.argmax(axis=-1), r_true.argmax(axis=-1)
left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred = left_click_pred.argmax(axis=-1), right_click_pred.argmax(axis=-1), w_pred.argmax(axis=-1), s_pred.argmax(axis=-1), a_pred.argmax(axis=-1), d_pred.argmax(axis=-1), shift_pred.argmax(axis=-1), e_pred.argmax(axis=-1), q_pred.argmax(axis=-1), one_pred.argmax(axis=-1), two_pred.argmax(axis=-1), r_pred.argmax(axis=-1)
"""
race_true, gender_true = race_true.argmax(axis=-1), gender_true.argmax(axis=-1)
race_pred, gender_pred = race_pred.argmax(axis=-1), gender_pred.argmax(axis=-1)
age_true = age_true * data_generator.max_age
age_pred = age_pred * data_generator.max_age"""

In [ ]:
x_true, y_true = x_true.astype(int), y_true.astype(int)

In [ ]:
x_true.shape

In [ ]:
x_true = x_true.reshape((len(x_true), 1))

x_pred = x_pred.reshape((len(x_true), 1))

In [ ]:
from sklearn.metrics import r2_score
print('R2 score for age: ', r2_score(x_true, x_pred))

In [ ]:
df.sample()['file'].values[0]

In [ ]:
sample = df.sample()
file = sample['file'].values[0] + '.jpeg'
im = Image.open(os.path.join("Screen_Capturing/Data/",file)).convert('RGB')
im = im.resize((IM_WIDTH, IM_HEIGHT))
im = np.array(im) / 255.0
images = []
images.append(im)

images = np.array(images)
file

In [ ]:
x_pred, y_pred, left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred = model.predict(images)

left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred = left_click_pred.argmax(axis=-1), right_click_pred.argmax(axis=-1), w_pred.argmax(axis=-1), s_pred.argmax(axis=-1), a_pred.argmax(axis=-1), d_pred.argmax(axis=-1), shift_pred.argmax(axis=-1), e_pred.argmax(axis=-1), q_pred.argmax(axis=-1), one_pred.argmax(axis=-1), two_pred.argmax(axis=-1), r_pred.argmax(axis=-1)
x_pred, y_pred = x_pred * max_x, y_pred * max_y

In [ ]:
x_pred, y_pred, left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred 

In [ ]:
def pred_frame(img):
    img = img.resize((IM_WIDTH, IM_HEIGHT))
    img = np.array(img) / 255.0
    images = []
    images.append(img)

    images = np.array(images)

    x_pred, y_pred, left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred = model.predict(images)

    left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred = left_click_pred.argmax(axis=-1), right_click_pred.argmax(axis=-1), w_pred.argmax(axis=-1), s_pred.argmax(axis=-1), a_pred.argmax(axis=-1), d_pred.argmax(axis=-1), shift_pred.argmax(axis=-1), e_pred.argmax(axis=-1), q_pred.argmax(axis=-1), one_pred.argmax(axis=-1), two_pred.argmax(axis=-1), r_pred.argmax(axis=-1)
    x_pred, y_pred = x_pred * max_x, y_pred * max_y

    return x_pred, y_pred, left_click_pred, right_click_pred, w_pred, s_pred, a_pred, d_pred, shift_pred, e_pred, q_pred, one_pred, two_pred, r_pred

In [ ]:
import win32gui
from PIL import ImageGrab
from time import time
import cv2 as cv

loop_time = time()
while(True):
    hwnd = win32gui.FindWindow(None, 'ARMOURY CRATE')
    rect = win32gui.GetWindowRect(hwnd)
    screenshot = ImageGrab.grab(bbox=(rect[0], rect[1], rect[2], rect[3]))

    """screenshot = np.array(screenshot)
    screenshot = cv.cvtColor(screenshot, cv.COLOR_RGB2BGR)"""

    print(pred_frame(screenshot))

    screenshot = np.array(screenshot)
    screenshot = cv.cvtColor(screenshot, cv.COLOR_RGB2BGR)

    cv.imshow('Computer Vision', screenshot)

    print('FPS {}'.format(1 / (time() - loop_time)))
    loop_time = time()
    if cv.waitKey(1) == ord('q'):
        break

cv.destroyAllWindows()